# Data Exploration

In [174]:
import pandas as pd
from pandas_profiling import ProfileReport
import datetime as dt
import matplotlib.pyplot as plt

In [12]:
data = pd.read_csv('../data/training.csv')

## Generate automated report

In [10]:
profile = data.profile_report(title="Pandas Profiling Report")

In [ ]:
profile.to_file('Pandas Profiling Report.html')

Reviewing this report, it appears that vehicle age, year, and various cost/price variables are highly correlated. That makes sense! There are various other correlating variable groups, as well.

## Examine missing variables

There are a couple variables that will be important to drop in the model:
    - highly missing variables (drop those with >50% missing. This is arbitrary - just capturing the ones I see as issues)
    - ID variables

In [44]:
missing_flag = data.isna().sum() > data.shape[0]*.5
drop_cols = list(missing_flag.index[missing_flag])
drop_cols

['PRIMEUNIT', 'AUCGUART']

In [38]:
# also dropping one unique ID and one ID that corresponds directly to descriptive definitions
data.filter(regex='ID|Id').nunique()

RefId          72983
WheelTypeID        4
dtype: int64

In [45]:
drop_cols.append('RefId')

In [62]:
data[['WheelType','WheelTypeID']].drop_duplicates()

,WheelType,WheelTypeID
0,Alloy,1.0
2,Covers,2.0
39,NaN,NaN
212,Special,3.0
2992,NaN,0.0


In [63]:
drop_cols.append('WheelTypeID')

In [64]:
drop_cols

['PRIMEUNIT', 'AUCGUART', 'RefId', 'WheelTypeID']

In [169]:
data_filtered = data.drop(drop_cols, axis = 1)

## Examine dependent variable

In [173]:
data_filtered['IsBadBuy'].value_counts()/data_filtered.shape[0]
# If we assign everyone 0, we would get 87.7% accuracy

0    0.877012
1    0.122988
Name: IsBadBuy, dtype: float64

## Derived variables

In [60]:
data_filtered.filter(regex='Date|Year|Age') # would anything else about purchdate matter? month?

,PurchDate,VehYear,VehicleAge
0,12/7/2009,2006,3
1,12/7/2009,2004,5
2,12/7/2009,2005,4
3,12/7/2009,2004,5
4,12/7/2009,2005,4
...,...,...,...
72978,12/2/2009,2001,8
72979,12/2/2009,2007,2
72980,12/2/2009,2005,4
72981,12/2/2009,2006,3


TODO: Skipping deriving variables for now, but this could be an area to explore

## Re-format variables

In [170]:
data_filtered['PurchDate'] = data_filtered['PurchDate'].apply(lambda x: dt.datetime.strptime(x, '%m/%d/%Y'))

In [171]:
data_filtered['PurchDateOrdinal'] = data_filtered['PurchDate'].map(dt.datetime.toordinal)
data_filtered = data_filtered.drop(['PurchDate'], axis = 1)

In [161]:
onehotted = pd.get_dummies(data_filtered).reset_index()

In [148]:
mmrcols = onehotted.filter(regex='MMR').columns.values

In [162]:
onehotted[mmrcols] = onehotted[mmrcols].apply(lambda x: np.array(x, dtype='float64'))

In [ ]:
# TODO: could do correlation plot of onehotted variables with target IsBadBuy since report only shows numeric

In [ ]:
# TODO: BYRNO should be categorical

## Impute with median

In [166]:
imputed = onehotted.fillna(onehotted.median()).drop(['index'], axis = 1)

## Write data

In [167]:
imputed.to_pickle('../data/processed.pkl')